## Glossário 

Este glossário define os termos de negócio, as fórmulas, os conceitos de governança e os componentes técnicos usados no pipeline ETL modularizado no Databricks.

### 1. Termos de Negócio e Métricas Financeiras
Estes termos são cruciais para a análise de Profit & Loss (P&L) e Unit Economics.

| Termo | Definição |
| :--- | :--- |
| Marketplace Two-Sided | Modelo de negócio da plataforma, que conecta dois grupos distintos e interdependentes (Lojas/Fornecedores e Motoristas/Logística) aos Clientes. |
| Unit Economics (UE) | Análise da rentabilidade de uma única transação (o pedido). O objetivo final do pipeline é calcular o Lucro Bruto Unitário. |
| Lucro Bruto Unitário | A métrica final do pipeline. Representa a Margem de Contribuição da plataforma em cada pedido após subtrair os COGS simulados da Receita Líquida Plataforma. |
| P&L (Profit & Loss) Unitário | Análise da Demonstração de Resultados (Receitas e Custos) focada no nível do pedido. | 
| GMV (Gross Merchandise Value) | Valor Bruto da Mercadoria. A receita total gerada por um pedido: subtotal_bruto + delivery_fee_cliente. |
| COGS (Cost of Goods Sold) |	Custos Variáveis diretos da transação: Repasse Logístico (motorista) e Custo Transacional (taxa de pagamento). |
| Receita Líquida Plataforma | Valor que a plataforma retém após a comissão da loja e antes dos repasses logísticos. |
| Margem de Contribuição | Sinônimo de Lucro Bruto Unitário no contexto do projeto. |

#### 2. Fórmulas Implementadas no Pipeline (Cálculo Silver)

As fórmulas refletem o cálculo simulado da Unit Economics baseado nas constantes de negócio (0.18, 0.70, 0.02).

| Métrica | Fórmula (Simulada no Silver) |
| :--- | :--- |
| Receita Líquida Plataforma | 	(subtotal_bruto * 0.18) + (delivery_fee_cliente * (1 - 0.70)) |
| COGS Logístico Simulado | delivery_fee_cliente * 0.70 |
| COGS Transacional Simulado | subtotal_bruto * 0.02 |
| Lucro Bruto Unitário | Receita Líquida Plataforma - COGS Logístico - COGS Transacional |

#### 3. Arquitetura e Plataforma

| Termo | Definição |
| :--- | :--- |
| Databricks | Plataforma Lakehouse que hospeda o pipeline, fornecendo um ambiente unificado para dados e análise. |
| Apache Spark | Motor de processamento de computação distribuída que executa todas as transformações ETL de forma escalável e em memória. |
| Delta Lake | Tecnologia que garante a confiabilidade (ACID properties) e otimizações de data warehousing (ZORDER, versionamento) para todas as tabelas. |
| ZORDER | Técnica de otimização de consulta no Delta Lake. Agrupa fisicamente dados no disco pelas colunas mais consultadas (ex: order_date_key, store_id) para acelerar o BI. |
| Star Schema | Modelo implementado na Camada Gold, usando a Tabela Fato (tbl_fato_delivery_gold) ligada às Dimensões (tbl_dim_store_gold, tbl_dim_time_gold). |

#### 4. Camadas e Data Quality (DQ)

| Camada / Componente | Função Principal e DQ Aplicada | Processo Chave |
| :--- | :--- | :--- |
| Bronze | Ingestão/Landing Zone. Dados brutos e imutáveis. Mínima padronização de schema. | 01_bronze_ingestion.ipynb |
| Silver | Transformação/Cálculo UE. Aplica Deduplicação (ROW_NUMBER), filtra pedidos não-entregues e implementa DQ de Negócio (cheque de nulos/negativos de entrada). | 02_silver_ue_calculation_dq.ipynb |
| Gold |	Consumo/BI. Camada final com dados modelados e otimizados para o Power BI. | 03_gold_dimensional_model.ipynb |
| Data Testing | Governança/Validação. Notebook de testes de unidade que garantem a integridade referencial e a validade das constantes após a modelagem Gold. | 04_Final_Quality_Checks.ipynb |
| Deduplicação (ROW_NUMBER) | Regra usada na Camada Silver para resolver o Fan-out e garantir que cada order_id tenha apenas uma linha final (relação 1:1). | silver_ue_calculation |

#### 5. Componentes de Governança e Log

| Tabela | Definição | Contexto |
| :--- | :--- | :--- |
| log_processamento | 	Tabela de auditoria que registra o status (sucesso/falha/alerta) e o tempo de execução de cada notebook do Job. | Orquestração |
| metricas_qualidade | Tabela que armazena os resultados numéricos dos DQ Checks (ex: percentual de pedidos não-rentáveis). | Monitoramento |
| Integridade Referencial | Teste de DQ que verifica se todo store_id na Fato Gold tem correspondência na Dimensão Loja. | final_data_testing |
| Validação de Constantes |	Teste de DQ que compara o valor lucro_bruto_unitario armazenado com um recálculo usando as premissas de negócio fixas, garantindo que a lógica não foi alterada. | final_data_testing |
| DBU (Databricks Unit) | Unidade de consumo computacional no Databricks, utilizada para cálculo de custos do pipeline. | Otimização |


